In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf
import time
import yfinance as yf

import os
from os import listdir
from os.path import isfile, join

In [3]:
#default Values
path =r"D:\\Personal\\Quantitative-Finance\\stocks\\"
S_year = 2020
S_month = 1
S_day = 2
S_date_str = f"{S_year}-{S_month}-{S_day}"
S_date_datetime = dt.datetime(S_year,S_month,S_day)

E_year = 2024
E_month = 6
E_day = 14
E_date_str = f"{E_year}-{E_month}-{E_day}"
E_date_datetime = dt.datetime(E_year,E_month,E_day)


In [4]:
# get stock file names in a list
files = [x for x in listdir(path) if isfile(join(path,x))]
tickers = [os.path.splitext(x)[0] for x in files]

In [18]:
# tickers,len(tickers)

In [5]:
# create dataframe from our list
stock_df = pd.DataFrame(tickers,columns=["tickers"])
# stock_df

In [6]:
# return dataframe from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv("D:\\Personal\\Quantitative-Finance\\stocks\\"+ticker+ ".csv")
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df

In [7]:
# Save dataframe to csv
def save_dataframe_to_csv(df,ticker):
    df.to_csv(path+ticker+".csv")

In [8]:
# delete unnamed columns in csv files
def delete_unnames_cols(df):
    df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    return df

In [9]:
# Add daily return to df
def add_daily_return(df,ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    df.to_csv("D:\\Personal\\Quantitative-Finance\\stocks\\"+ticker+".csv")
    return df

In [11]:
# stock_a = get_df_from_csv(tickers[0])
# stock_a
for ticker in tickers:
    stock_a = get_df_from_csv(ticker)
    add_daily_return(stock_a,ticker)
# add_daily_return(stock_a,tickers[0])
# stock_a

In [ ]:
# return on investment over time
def get_roi_defined_time(df):
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_date_str]['Adj Close'][0]
    end_val = df[df['Date'] == E_date_str]['Adj Close'][0]
    print("Initial Price:",start_val)
    print("Final Price:",end_val)
    roi = (end_val - start_val) / start_val
    return roi

In [ ]:
# get coefficient of variance
def get_cov(stock_df):
    for stock in stock_df:
        mean,sd = stock_df['Adj Close'].mean(),stock_df['Adj Close'].std()
        cov = sd / mean
        return cov

In [16]:
stock_a = get_df_from_csv(tickers[0])
stock_a

# add_daily_return(stock_a,tickers[0])
# stock_a

# stock_a = delete_unnames_cols(stock_a)
# stock_a

# save_dataframe_to_csv(stock_a, tickers[1])

File doesn't exist


In [ ]:
# Merge Multiple stock in on df
def merge_df_by_column_name(col_name,*tickers):
    
    mult_df = pd.DataFrame()
    start = S_date_datetime
    end = E_date_datetime
    
    for x in tickers:
        mult_df[x] = get_df_from_csv(x)[col_name]
    return mult_df

In [ ]:
port_list = ["FB","NEM"]
port_df = merge_df_by_column_name('daily_return',*port_list)

In [ ]:
port_df.corr()

,FB,NEM
FB,1.000000,0.112418
NEM,0.112418,1.000000


In [ ]:
price_df = merge_df_by_column_name('Adj Close',*["FB","NEM"])

In [ ]:
price_df.corr()

,FB,NEM
FB,1.000000,0.878595
NEM,0.878595,1.000000


In [ ]:
price_df

,FB,NEM
0,116.860001,31.397280
1,118.690002,31.696466
2,120.669998,33.156181
3,123.410004,32.113529
4,124.900002,32.059132
...,...,...
1161,363.179993,58.840000
1162,366.559998,58.880001
1163,358.450012,58.860001
1164,355.450012,56.910000


In [ ]:
fb_wt = 355 / 411.5
new_wt = 56.5 / 411.5

wts = np.array([fb_wt,new_wt])
port_var = np.dot(wts.T, np.dot(port_df.cov() * 253, wts))
print("Portfolio Var :", port_var)
print("FB Var: ",port_df["FB"].var()*253)
print("NEM Var: ",port_df["NEM"].var()*253)

Portfolio Var : 0.08810500277812469
FB Var:  0.11213621430466478
NEM Var:  0.09826110042532281
